In [ ]:
import subprocess

infile="/mnt/disk1/tapearchive/A4/211008_0016.wav" 
outfile = "/home/caiwan/test3.flac"

filter_stack = ";".join([
    "acopy[out]",
    "[out]channelsplit=channel_layout=stereo:channels=FL[out]"
    "[out]dynaudnorm=framelen=1000:maxgain=3:coupling=false[out]"
])

ffmpeg_commnad=f"ffmpeg -y -i {infile} -filter_complex {filter_stack} -map [out] {outfile}"

print(ffmpeg_commnad.split())

ffmpeg_process = subprocess.Popen(ffmpeg_commnad.split(), stdout=subprocess.PIPE, stdin=subprocess.PIPE)

while ffmpeg_process.poll() is None:
    try:
        stdout_data, stderr_data = ffmpeg_process.communicate(timeout=30)
    except subprocess.TimeoutExpired:
        ffmpeg_process.kill()
        stdout_data, stderr_data = ffmpeg_process.communicate()
    print(stdout_data, stderr_data)

In [ ]:
import sys
import logging


LOGGER = logging.getLogger()
LOGGER.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stderr)

formatter = logging.Formatter('%(threadName)s: %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

LOGGER.handlers = [handler]

In [ ]:
import waiting
from contextlib import ExitStack

from tq.task_dispacher import LocalTaskQueue, TaskDispatcher
from tq.job_system import JobManager

from tapearchive.models.records import ChannelMode
from tapearchive.workflow.audio_convert import ConvertAudio, AudioConverterHandler

# Do task
task_queue = LocalTaskQueue()
with ExitStack() as stack:
    job_manager = stack.enter_context(JobManager())
    dispatcher = stack.enter_context(TaskDispatcher(task_queue, job_manager))

    audio_converter_handler = AudioConverterHandler()
    dispatcher.register_task_handler(audio_converter_handler)

    logging.debug(dispatcher.task_handlers)

    dispatcher.post_task(ConvertAudio(
        source="/mnt/disk1/tapearchive/B4/211014_0050.wav",
        source_channel=ChannelMode.LEFT,
        target = "/home/caiwan/test_L.flac",
    ))

    dispatcher.post_task(ConvertAudio(
        source="/mnt/disk1/tapearchive/B4/211014_0050.wav",
        source_channel=ChannelMode.RIGHT,
        target = "/home/caiwan/test_R.flac",
    ))

    waiting.wait(lambda: len(audio_converter_handler.tasks_finished)==2, sleep_seconds=1, timeout_seconds=120)

    logging.info(audio_converter_handler.tasks_finished)